dependencies </n>
Env :: Forecastinit
Python=3.11.5

Libraries

In [336]:
# General 
import pandas as pd
import numpy as np

# sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# skforecast 
from skforecast.datasets import load_demo_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

# ML Models 
from xgboost import XGBRegressor

Input

In [337]:
Historical_data_org = pd.read_csv('Input/Actual.csv')
TimeKey_org = pd.read_csv('Input/Time.csv')
Forecast_level = ['Version.[Version Name]', 'Channel.[Channel]', 'Account.[Account]',
       'PnL.[PnL]', 'Demand Domain.[Demand Domain]', 'Region.[Region]',
       'Location.[Location]', 'Time.[Planning Month]', 'Item.[Item]']
Time_column = ['Time.[Planning Month]']
Historical_data_column = ['Actual']
#Date should be in format of time key column, which is available in TimeKey file 
Time_key_column_name = "Time.[PlanningMonthKey]"
Time_key_date_format = "%m/%d/%Y %I:%M:%S %p"
Historic_start_date = "M09-20" 
Historic_end_date = "M12-22"
Forecast_start_date = "M01-23"
Forecast_end_date = "M03-23"
drivers = ['driver_Month']
models = ["Random Forest", "XG Boost"]
Hypertunning = False


In [338]:
# lags used in grid search hyper tunning 
lags_grid = [2,3,6,[1,2,3],[1,2,3,6],[3,6],[1,2,3,6]]
# lags_grid = [2,6]

# parameters used in grid search hyper tunning 
param_grid = {
    'n_estimators': [50, 100, 200, 500, 1000, 1500],
    'max_depth': [2, 5, 8, 10, 12, 15]
}


Custom Functions

In [386]:
# Data filter based on start and end date, then split in X and y based on provided columns 
# split(historical data frame contains drivers and actual, start date , end date, X columns in list, y column in string )
def split(data,Historic_start_date_key,Historic_end_date_key,Forecast_start_date_key,Forecast_end_date_key,X_cols,y_col):
    data = data[(data['TimeKey']>=Historic_start_date_key)&(data['TimeKey']<=Forecast_end_date_key)].reset_index(drop=True)

    # train/test 
    train = data = data[(data['TimeKey']>=Historic_start_date_key)&(data['TimeKey']<=Historic_end_date_key)]
    test = data = data[(data['TimeKey']>Historic_end_date_key)&(data['TimeKey']<=Forecast_end_date_key)]
    
    X_train = data[['key','TimeKey']+X_cols]
    y = data[['TimeKey']+[y_col]]
    return X, y

In [340]:
# fit the model 
# if Hypertunning is turned on fit will be in grid search else normal fit with default values will return 
def skforecastfit(forecaster,y_train_loc,param_grid,lags_grid):
    results = grid_search_forecaster(
              forecaster         = forecaster,
              y                  = y_train_loc,
              param_grid         = param_grid,
              lags_grid          = lags_grid,
              steps              = 12,
              refit              = False,
              metric             = 'mean_squared_error',
              initial_train_size = len(y_train)-1,
              fixed_train_size   = False,
              return_best        = True,
              n_jobs             = 'auto',
              verbose            = False,
              show_progress      = True
          )   

In [341]:
# SK Forecast ML Models
# skforecastpredict(models in list, X train data frame, y train data frame, X test data frame ) 
def skforecastpredict(models, X_train, y_train, X_test,Hypertunning,param_grid,lags_grid):
    # Predicted output data frame 
    Output = pd.DataFrame()

    # converting data frame to series, as sk forecast y accept series 
    y_train_loc = y_train[y_train.columns.values[0]]
    
    # from sklearn.preprocessing import FunctionTransformer
    # transformer_y = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

    # Random Forest ========================================================================================================================= 
    if "Random Forest" in models:
        randomforestforecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = [2]
             )
        if Hypertunning is True:
            skforecastfit(randomforestforecaster,y_train_loc,param_grid,lags_grid)
        else:
            randomforestforecaster.fit(y= y_train_loc)
        y_hat_randomforest = randomforestforecaster.predict_interval(steps=3).reset_index(drop=True)
        y_hat_randomforest.columns = ['Random Forest Y_hat', 'Random Forest Lower Bound', 'Random Forest Upper Bound']
        Output = pd.concat([Output, y_hat_randomforest], axis=1)
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    
    #XG Boost ================================================================================================================================
    if "XG Boost" in models:
        xgboostForecaster = ForecasterAutoreg(
                regressor= XGBRegressor(random_state = 213),
                lags= [1,2]
            )      
        if Hypertunning is True:
            skforecastfit(xgboostForecaster,y_train_loc,param_grid,lags_grid)
        else:
            xgboostForecaster.fit(y=y_train_loc)
        y_hat_xgboost = xgboostForecaster.predict_interval(steps=3).reset_index(drop=True)
        y_hat_xgboost.columns = ['XG Boost Y_hat', 'XG Boost Lower Bound', 'XG Boost Upper Bound']
        Output = pd.concat([Output, y_hat_xgboost], axis=1)
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
        
    return Output

Processing Input

In [342]:
Forecast_level_minus_time = Forecast_level.copy()
Forecast_level_minus_time.remove(Time_column[0])
Forecast_level_minus_time

['Version.[Version Name]',
 'Channel.[Channel]',
 'Account.[Account]',
 'PnL.[PnL]',
 'Demand Domain.[Demand Domain]',
 'Region.[Region]',
 'Location.[Location]',
 'Item.[Item]']

Copy Input 

In [343]:
Historical_data = Historical_data_org.copy(deep=True)
TimeKey = TimeKey_org.copy(deep=True)
TimeKey['TimeKey'] = pd.to_datetime(TimeKey[Time_key_column_name], format=Time_key_date_format)

In [344]:
# calculating Historic_end_date_key
Historic_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_start_date]['TimeKey'].values[0])
Historic_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_end_date]['TimeKey'].values[0])
Forecast_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_start_date]['TimeKey'].values[0])
Forecast_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_end_date]['TimeKey'].values[0])

Processing Copy Input

In [345]:
Historical_data[Historical_data_column] = Historical_data[Historical_data_column].astype(float)


Creating Key in Historical Data

In [346]:
# creating keys 
Historical_data['key'] = Historical_data[Forecast_level_minus_time].astype(str).agg("__ MDJoinner__".join, axis=1)
# dropping columns, which is already present in keys 
Historical_data.drop(Forecast_level_minus_time,axis=1,inplace=True) 
Historical_data.head(2)

,Time.[Planning Month],Actual,key
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...


Custom Filter

In [347]:
Historical_data = Historical_data[Historical_data['key']=='CurrentWorkingView__ MDJoinner__B2B__ MDJoinner__AMO__ MDJoinner__DP__ MDJoinner__DP__ MDJoinner__ShipTo1__ MDJoinner__DP__ MDJoinner__Loctite 248 19g Stick']
# Historical_data['key'].values

Merging Time Key in Historical Data

In [348]:
Historical_data = pd.merge(Historical_data,TimeKey[Time_column+["TimeKey"]],on=Time_column,how='left')
Historical_data.head(2)

,Time.[Planning Month],Actual,key,TimeKey
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02


Adding features

In [349]:
# Creating drivers 
Historical_data['driver_Month'] = Historical_data['TimeKey'].dt.month
Historical_data.head()

,Time.[Planning Month],Actual,key,TimeKey,driver_Month
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05,7
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02,8
2,M09-20,462.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-30,8
3,M10-20,174.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-10-04,10
4,M11-20,179.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-11-01,11


Train and Test Data

In [387]:
X_train,y_train,X_test,y_test = split(Historical_data, Historic_start_date_key,Historic_end_date_key,Forecast_start_date_key,Forecast_end_date_key,drivers,Historical_data_column[0])
# X_test,y_test = split(Historical_data, Forecast_start_date_key, Forecast_end_date_key, drivers, Historical_data_column[0])

Prediction

In [335]:
Hypertunning=True
models = ["~Random Forest", "XG Boost"]
rf = skforecastpredict(models,X_train, y_train, X_test, Hypertunning, param_grid, lags_grid)

Number of models compared: 252.


lags grid:   0%|          | 0/7 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:877: UserWarning: `last_window` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(


XGBoostError: [14:37:02] /croot/xgboost-split_1675457761144/work/src/c_api/../data/array_interface.h:127: Check failed: typestr.size() == 3 || typestr.size() == 4: `typestr' should be of format <endian><type><size of type in bytes>.
Stack trace:
  [bt] (0) /home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/libxgboost.so(+0xc01d4) [0x7ff5eb6801d4]
  [bt] (1) /home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/libxgboost.so(+0xd3dfc) [0x7ff5eb693dfc]
  [bt] (2) /home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/libxgboost.so(+0xe5758) [0x7ff5eb6a5758]
  [bt] (3) /home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/libxgboost.so(xgboost::data::DMatrixProxy::SetArrayData(char const*)+0x131) [0x7ff5eb7deb21]
  [bt] (4) /home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/libxgboost.so(XGBoosterPredictFromDense+0xcc) [0x7ff5eb69f14c]
  [bt] (5) /home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/lib-dynload/../../libffi.so.8(+0xa052) [0x7f

In [388]:
X_train

,key,TimeKey,driver_Month
2,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-30,8
3,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-10-04,10
4,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-11-01,11
5,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-11-29,11
6,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2021-01-03,1
7,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2021-01-31,1
8,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2021-02-28,2
9,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2021-04-04,4
10,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2021-05-02,5
11,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2021-05-30,5


In [384]:
y_train

,TimeKey,Actual
0,2020-08-30,462.0
1,2020-10-04,174.0
2,2020-11-01,179.0
3,2020-11-29,231.0
4,2021-01-03,126.0
5,2021-01-31,212.0
6,2021-02-28,231.0
7,2021-04-04,135.0
8,2021-05-02,182.0
9,2021-05-30,288.0


In [385]:
X_test

,key,TimeKey,driver_Month
0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2023-01-01,1
1,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2023-01-29,1
2,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2023-02-26,2


In [354]:
y_test

,TimeKey,Actual
30,2023-01-01,209.0
31,2023-01-29,223.0
32,2023-02-26,333.0


In [ ]:
Historical_data[drivers].head(28)

,driver_Month
0,7
1,8
2,8
3,10
4,11
5,11
6,1
7,1
8,2
9,4


In [ ]:
Historical_data[drivers].tail(8)

,driver_Month
28,10
29,11
30,1
31,1
32,2
33,4
34,4
35,5


,key,driver_Month
0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,1
1,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,1
2,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2


In [381]:
# Grid search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 10 # Placeholder, the value will be overwritten
             )

# Lags used as predictors
lags_grid = [2,3,6,[1,2,3],[1,2,3,6],[3,6],[1,2,3,6]]

lags_grid = [2,3]
# Regressor hyperparameters
param_grid = {
    'n_estimators': [50, 100, 200, 500, 1000, 1500],
    'max_depth': [2, 5, 8, 10, 12, 15]
}

forecaster.fit(y=y_train[Historical_data_column[0]],exog=X_train[drivers])
forecaster.predict_interval(steps=3,exog=X_test[drivers])
# lags_grid = [2, 10, [1, 2, 3, 20]]

# Regressor hyperparameters
# param_grid = {
#     'n_estimators': [50, 100],
#     'max_depth': [5, 10, 15]
# }



ValueError: To make predictions `exog` must start one step ahead of `last_window`.
    `last_window` ends at : 27.
    `exog` starts at      : 0.
     Expected index       : 28.

In [260]:
forecaster
X_train[drivers]
X_test[drivers]

,driver_Month
TimeKey,
2023-01-01,1
2023-01-29,1
2023-02-26,2


In [256]:
X_test

,key,driver_Month
TimeKey,,
2023-01-01,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,1
2023-01-29,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,1
2023-02-26,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2


In [249]:
y_test

,Actual
TimeKey,
2023-01-01,209.0
2023-01-29,223.0
2023-02-26,333.0


In [245]:
X_train

,key,driver_Month
TimeKey,,
2020-08-30,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,8
2020-10-04,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,10
2020-11-01,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,11
2020-11-29,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,11
2021-01-03,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,1
2021-01-31,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,1
2021-02-28,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2
2021-04-04,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,4
2021-05-02,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,5


In [248]:
y_train

,Actual
TimeKey,
2020-08-30,462.0
2020-10-04,174.0
2020-11-01,179.0
2020-11-29,231.0
2021-01-03,126.0
2021-01-31,212.0
2021-02-28,231.0
2021-04-04,135.0
2021-05-02,182.0


In [159]:
rf
yhat = rf.rename(columns={'Random Forest Y_hat':'pred','XG Boost Y_hat':'pred'})

yhat

,pred,XG Boost Lower Bound,XG Boost Upper Bound
0,283.795776,283.788559,283.857056
1,426.195557,426.187851,426.256836
2,693.000244,692.992538,693.061523


In [144]:
yhat = forecaster.predict_interval(3)
yhat.reset_index(inplace=True,drop=True)
yhat

NotFittedError: This Forecaster instance is not fitted yet. Call `fit` with appropriate arguments before using predict.

In [160]:
y_test.reset_index(inplace=True,drop=True)
y_test


,Actual
0,209.0
1,223.0
2,333.0


In [161]:
yhat = pd.merge(yhat[['pred']],y_test,left_index=True,right_index=True,how='outer')
yhat

,pred,Actual
0,283.795776,209.0
1,426.195557,223.0
2,693.000244,333.0


In [162]:
yhat['at'] = 'all'
yhat['diff'] = abs(yhat['pred'] - yhat['Actual'])
yhat = yhat.groupby(['at'],as_index=False)[['diff','Actual']].sum()
yhat

,at,diff,Actual
0,all,637.991577,765.0


In [163]:
print("Accuracy",100-yhat['diff']/yhat['Actual']*100)


Accuracy 0    16.602408
dtype: float64


In [269]:
url = (
    'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
    'data/h2o_exog.csv'
)
data = pd.read_csv(
            url, sep=',', header=0, names=['datetime', 'y', 'exog_1', 'exog_2']
       )

In [270]:
data

,datetime,y,exog_1,exog_2
0,1992-04-01,0.379808,0.958792,1.166029
1,1992-05-01,0.361801,0.951993,1.117859
2,1992-06-01,0.410534,0.952955,1.067942
3,1992-07-01,0.483389,0.958078,1.097376
4,1992-08-01,0.475463,0.956370,1.122199
...,...,...,...,...
190,2008-02-01,0.761822,1.515840,1.786373
191,2008-03-01,0.649435,1.506258,1.694264
192,2008-04-01,0.827887,1.505253,1.627135
193,2008-05-01,0.816255,1.491464,1.555068


In [271]:
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data.sort_index()
data

,y,exog_1,exog_2
datetime,,,
1992-04-01,0.379808,0.958792,1.166029
1992-05-01,0.361801,0.951993,1.117859
1992-06-01,0.410534,0.952955,1.067942
1992-07-01,0.483389,0.958078,1.097376
1992-08-01,0.475463,0.956370,1.122199
...,...,...,...
2008-02-01,0.761822,1.515840,1.786373
2008-03-01,0.649435,1.506258,1.694264
2008-04-01,0.827887,1.505253,1.627135


In [272]:
steps = 36
data_train = data.iloc[:-steps, :]
data_test  = data.iloc[-steps:, :]

In [273]:
data_train

,y,exog_1,exog_2
datetime,,,
1992-04-01,0.379808,0.958792,1.166029
1992-05-01,0.361801,0.951993,1.117859
1992-06-01,0.410534,0.952955,1.067942
1992-07-01,0.483389,0.958078,1.097376
1992-08-01,0.475463,0.956370,1.122199
...,...,...,...
2005-02-01,0.597639,1.520544,1.784523
2005-03-01,0.652590,1.506290,1.678839
2005-04-01,0.670505,1.487660,1.569732


In [274]:
data_test

,y,exog_1,exog_2
datetime,,,
2005-07-01,0.874336,1.415756,1.446988
2005-08-01,1.006497,1.398304,1.517770
2005-09-01,1.094736,1.386174,1.602616
2005-10-01,1.027043,1.363155,1.668975
2005-11-01,1.149232,1.361009,1.730369
2005-12-01,1.160712,1.417316,1.787644
2006-01-01,1.230691,1.475126,1.832483
2006-02-01,0.587135,1.466789,1.730963
2006-03-01,0.706959,1.467960,1.666946


In [275]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 36
             )

forecaster.fit(
    y    = data_train['y'],
    exog = data_train[['exog_1', 'exog_2']]
)

forecaster

ForecasterAutoreg 
Regressor: RandomForestRegressor(random_state=123) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36] 
Transformer for y: None 
Transformer for exog: None 
Window size: 36 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['exog_1', 'exog_2'] 
Training range: [Timestamp('1992-04-01 00:00:00'), Timestamp('2005-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_st

In [279]:
# Predict
# ==============================================================================
steps = 36
predictions = forecaster.predict(
                  steps = 37,
                  exog = data_test[['exog_1', 'exog_2']]
              )

predictions

ValueError: `exog` must have at least as many values as the distance to the maximum step predicted, 37.